In [1]:
from typing import List, Dict
import pandas as pd
import sys
sys.path.append("../")
sys.path.append("../src/")
from tqdm.notebook import tqdm
from src.engine.morph import Morph
from src.engine.utils import fts_text_processing_passage, embedding_text_processing_passage
from src.engine.embedding import Embedding

morph_model = Morph()

In [2]:
data = pd.read_parquet("../data/data_raw.pq")
data.rename(columns={col:"raw_" + "_".join(col.split()) for col in data.columns.values if col not in ["index orig", "link"]}, inplace=True)
data.rename(columns={"link": "video_url"}, inplace=True)

data["_id"] = data["video_url"]
data.head()

,index orig,video_url,raw_description,raw_transcription_whisper_v3,raw_image_captioning,raw_title_shazam_transliterated,raw_subtitle_shazam_transliterated,raw_title_shazam,raw_subtitle_shazam,_id
0,5000,https://cdn-st.rutubelist.ru/media/67/86/ebba7...,"Sejak saat itu, senyumku jadi palsu-----------...",もう! 雉真のバカ!雉真のせいで勉強できない!せっかくしたのに雉真があんな態度だから思い出す...,an anime character with blue hair and blue eyes,Идеально,Миллер Джейми,Here's Your Perfect,Jamie Miller,https://cdn-st.rutubelist.ru/media/67/86/ebba7...
1,5001,https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...,#красивыедевушки #азиатки #boobs #грудь,Субтитры сделал DimaTorzok,a woman with blue skin and dark hair,Джи,"Я не знаю, что делать.",走路像只鸭(DJ咚鼓版),"黑暗萝莉, 孙大佛爷, 索菲亚, 沈子凡 & 社长姐姐",https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...
2,5002,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...,"#boobs , #bigass , #girls , #pussy , #еда , #г...","Шаман Субтитры сделал DimaTorzok Краду, краду...",a man with blue hair and a microphone,Русский король Шаман Версия,Король Шаман,Shaman King (Russian Version),SHAMAN KING,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...
3,5003,https://cdn-st.rutubelist.ru/media/67/59/68236...,#образ #lookbook #показ #неделямоды #капсула #...,Так правильно выбрать актуальное худио. Ставь...,a person wearing gloves and a white shirt,Говорить о Луне,Бруно Марс,Talking to the Moon,Bruno Mars,https://cdn-st.rutubelist.ru/media/67/59/68236...
4,5004,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...,#уходзакожей #уходзасобой #бьютирутина #бьюти ...,ТОП 5 стайлингов из масс-маркета ТОП 5 стайли...,a tube of tat hair care product on a white ba...,Горячий,2 Команды в прямом эфире,Me So Horny,The 2 Live Crew,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...


In [3]:
# for fts
full_text = []
text_hashtags = []
song_name = []
song_author = []
song_name_transliterated = []
song_author_transliterated = []
video_hashtags = []
audio_hashtags = []
audio_transcription = []
# for embedding
text_to_embedds = []

for _, row in tqdm(data.iterrows(),):
    text_to_embedd: str = embedding_text_processing_passage(
        morph=morph_model,
        raw_description=row.raw_description,
        raw_audio_transcription=row.raw_transcription_whisper_v3,
        raw_video_hashtags=row.raw_image_captioning,
    )

    text_to_fts: Dict[str, str] = fts_text_processing_passage(
        morph=morph_model,
        raw_description=row.raw_description,
        raw_song_name=row.raw_title_shazam,
        raw_song_author=row.raw_subtitle_shazam,
        raw_song_name_transliterated=row.raw_title_shazam_transliterated,
        raw_song_author_transliterated=row.raw_subtitle_shazam_transliterated,
        raw_audio_transcription=row.raw_transcription_whisper_v3,
        raw_video_hashtags=row.raw_image_captioning,
    )

    text_to_embedds.append(text_to_embedd)
    full_text.append(text_to_fts["full_text"])
    text_hashtags.append(text_to_fts["clean_description"])
    song_name.append(text_to_fts["clean_song_name"])
    song_author.append(text_to_fts["clean_song_author"])
    song_name_transliterated.append(text_to_fts["clean_song_name_transliterated"])
    song_author_transliterated.append(text_to_fts["clean_song_author_transliterated"])
    video_hashtags.append(text_to_fts["clean_video_hashtags"])
    audio_hashtags.append(text_to_fts["clean_audio_hashtags"])
    audio_transcription.append(text_to_fts["clean_audio_transcription"])

0it [00:00, ?it/s]

In [4]:
data["text_to_embedd"] = text_to_embedds
data["full_text"] = full_text
data["text_hashtags"] = text_hashtags
data["song_name"] = song_name
data["song_author"] = song_author
data["song_name_transliterated"] = song_name_transliterated
data["song_author_transliterated"] = song_author_transliterated
data["video_hashtags"] = video_hashtags
data["audio_hashtags"] = audio_hashtags
data["audio_transcription"] = audio_transcription

In [6]:
embedding = Embedding(device="cuda:0")

bs = 100
text_embeddings: List[List[float]] = []
for i in tqdm(range(0, len(text_to_embedds), bs)):
    batch = text_to_embedds[i:i+bs]
    text_embeddings.extend(embedding(texts=batch))

  0%|          | 0/400 [00:00<?, ?it/s]

In [7]:
data["embedding"] = text_embeddings

In [10]:
data.to_parquet("../data/data_postproc.pq")

In [14]:
data.head()

,index orig,video_url,raw_description,raw_transcription_whisper_v3,raw_image_captioning,raw_title_shazam_transliterated,raw_subtitle_shazam_transliterated,raw_title_shazam,raw_subtitle_shazam,_id,...,full_text,text_hashtags,song_name,song_author,song_name_transliterated,song_author_transliterated,video_hashtags,audio_hashtags,audio_transcription,embedding
0,5000,https://cdn-st.rutubelist.ru/media/67/86/ebba7...,"Sejak saat itu, senyumku jadi palsu-----------...",もう! 雉真のバカ!雉真のせいで勉強できない!せっかくしたのに雉真があんな態度だから思い出す...,an anime character with blue hair and blue eyes,Идеально,Миллер Джейми,Here's Your Perfect,Jamie Miller,https://cdn-st.rutubelist.ru/media/67/86/ebba7...,...,sejak saat itu senyumku jadi palsuanime midara...,sejak saat itu senyumku jadi palsuanime midara...,heres your perfect,jamie miller,идеально,миллер джейми,blue eyes anime hair character,,,"[-0.01246562134474516, 0.005366824101656675, -..."
1,5001,https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...,#красивыедевушки #азиатки #boobs #грудь,Субтитры сделал DimaTorzok,a woman with blue skin and dark hair,Джи,"Я не знаю, что делать.",走路像只鸭(DJ咚鼓版),"黑暗萝莉, 孙大佛爷, 索菲亚, 沈子凡 & 社长姐姐",https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...,...,красивыедевушки азиатки boobs грудь dj джи я ...,красивыедевушки азиатки boobs грудь,dj,,джи,я не знаю что делать,woman hair blue dark skin,,субтитры сделал dimatorzok,"[-0.00865133386105299, 0.012044253759086132, -..."
2,5002,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...,"#boobs , #bigass , #girls , #pussy , #еда , #г...","Шаман Субтитры сделал DimaTorzok Краду, краду...",a man with blue hair and a microphone,Русский король Шаман Версия,Король Шаман,Shaman King (Russian Version),SHAMAN KING,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...,...,boobs bigass girls pussy еда готовка рецепт ку...,boobs bigass girls pussy еда готовка рецепт ку...,shaman king russian version,shaman king,русский король шаман версия,король шаман,man blue microphone hair,,шаман субтитры сделал dimatorzok краду краду к...,"[0.018505407497286797, 0.02669624425470829, -0..."
3,5003,https://cdn-st.rutubelist.ru/media/67/59/68236...,#образ #lookbook #показ #неделямоды #капсула #...,Так правильно выбрать актуальное худио. Ставь...,a person wearing gloves and a white shirt,Говорить о Луне,Бруно Марс,Talking to the Moon,Bruno Mars,https://cdn-st.rutubelist.ru/media/67/59/68236...,...,образ lookbook показ неделямоды капсула ошибки...,образ lookbook показ неделямоды капсула ошибки...,talking to the moon,bruno mars,говорить о луне,бруно марс,white wearing shirt gloves person,лайк качество капюшон главное бедра фурнитуры ...,так правильно выбрать актуальное худио ставь л...,"[-0.006415102165192366, 0.02334967814385891, -..."
4,5004,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...,#уходзакожей #уходзасобой #бьютирутина #бьюти ...,ТОП 5 стайлингов из масс-маркета ТОП 5 стайли...,a tube of tat hair care product on a white ba...,Горячий,2 Команды в прямом эфире,Me So Horny,The 2 Live Crew,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...,...,уходзакожей уходзасобой бьютирутина бьюти ного...,уходзакожей уходзасобой бьютирутина бьюти ного...,me so horny,the 2 live crew,горячий,2 команды в прямом эфире,tube background white tat product care hair,,топ 5 стайлингов из массмаркета топ 5 стайлинг...,"[-0.0064900657162070274, 0.023763718083500862,..."


In [11]:
from src.utils import create_documents_jsonl

In [13]:
create_documents_jsonl(data=data, path_to_save="../data/documents.jsonl")